In [12]:
import requests
import numpy as np
import pandas as pd
import datetime as dt
from pandas.io.json import json_normalize
from psaw import PushshiftAPI
import praw
import pickle

In [13]:
pd.set_option('display.max_columns', 100)  # or 1000
pd.set_option('display.max_rows', 100)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [3]:
reddit = praw.Reddit(client_id='sIF2FculBPoFMg',
                     client_secret='ONlvY9ziXYVBhTeMvp1y4yP4Fg4',
                     user_agent='billyisnotthegoat')

api = PushshiftAPI(reddit)

In [4]:

before_date = int(dt.datetime(2019, 11, 1).timestamp())
after_date = int(dt.datetime(2017, 3, 1).timestamp())

gen = api.search_submissions( after=after_date, before=before_date, subreddit='Random_Acts_Of_Pizza')

## Get raw submissions from Random Acts of Pizza sub

In [ ]:
objects = list(gen)

kept = []

count = 0


for idx, obj in enumerate(objects):
    title = obj.title.lower()
    text = obj.selftext
    url = obj.url
    flair = obj.link_flair_text
 
        
    if not obj.author:
        continue
    
    if "request]" not in title or text == "[removed]" or text == "[deleted]":
        continue
        
    if flair == "No Longer Needed":
        continue
        
    if flair == "Fulfilled":
        count += 1
        
    kept.append(obj)


In [15]:
len(objects), count, len(kept)

(41880, 2673, 4754)

In [16]:
count / len(kept)

0.562263357172907

In [17]:
with open('submissions.pkl','wb') as f:
    pickle.dump(kept, f)

In [4]:
submissions = None
with open('submissions.pkl','rb') as f:
    submissions = pickle.load(f)

In [19]:
kept == submissions

True

## Construct necessary data for dataset

In [5]:
GiversBotId = 'np6d0'
# def getAllComments(submission, verbose=True):
#     comments = submission.comments
#     commentsList = []
    
#     def getSubComments(comment, verbose=True):
#         nonlocal commentsList
#         commentsList.append(comment)
#         if not hasattr(comment, "replies"):
#             replies = comment.comments()
#             if verbose: print("fetching (" + str(len(commentsList)) + " comments fetched total)")
#         else:
#             replies = comment.replies
#         for child in replies:
#             getSubComments(child, verbose=verbose)
    
#     for comment in comments:
#         getSubComments(comment,verbose=verbose)
#     return commentsList


def getSubComments(comment, subComments, verbose=True):
    if isinstance(comment, praw.models.reddit.comment.Comment):
        subComments.append(comment)
    if not hasattr(comment, "replies"):
        replies = comment.comments()
        if verbose: print("fetching (" + str(len(subComments)) + " comments fetched total)")
    else:
        replies = comment.replies
    for child in replies:
        getSubComments(child, subComments, verbose=verbose)


def construct_features(idx): #index in submissions
        submission = submissions[idx]
        print ("STARTING...", idx)
        d = {}
        
        d['request_id'] = submission.id
        d['request_number_of_comments_at_retrieval'] = submission.num_comments
        d['request_text'] = submission.selftext.lower()
        d['request_title'] = submission.title
       
        d['number_of_downvotes_of_request_at_retrieval'] = submission.downs
        d['number_of_upvotes_of_request_at_retrieval'] = submission.ups
    
        if submission.edited == False:
            d['post_was_edited'] = False
        else:
            d['post_was_edited'] = True
            
       
        
        d['requester_account_age_in_days_at_request'] = divmod(submission.created_utc - submission.author.created_utc, 86400)[0]
       
        d['request_url'] = submission.url
    
        
        try:
            requester_created_utc = submission.author.created_utc
        except: #if created_utc is not an attribute, the author has been suspended
            return

        
        requester_subreddits_at_request = set()
    
        for author_comment in submission.author.comments.top('all'):
            if author_comment.subreddit.created_utc < submission.created_utc:
                requester_subreddits_at_request.add(author_comment.subreddit.display_name)
    
        for author_sub in submission.author.submissions.top('all'):
            if author_sub.subreddit.created_utc < submission.created_utc:
                requester_subreddits_at_request.add(author_sub.subreddit.display_name)
                
        top_level_comments = list(submission.comments)
        
        d['requester_num_pizza_received_at_request'] = None
        d['requester_num_pizza_given_at_request'] = None
        d['requester_num_pizza_related_posts_at_request'] = None
        d['requester_num_pizza_related_comments_at_request'] = None
        
        d['giver_username'] = None
        d['giver_user_flair'] = None
        giver_redditor = None
        
        for tl_comment in top_level_comments:
            try:
                if tl_comment.author.id == GiversBotId:
                    if "* **Received" in tl_comment.body:
                        info_lines = tl_comment.body.splitlines()
                        d['requester_num_pizza_received_at_request'] = int(info_lines[0].split()[2] )
                        d['requester_num_pizza_given_at_request'] = int(info_lines[1].split()[2] )
                        d['requester_num_pizza_related_posts_at_request'] = int(info_lines[2].split()[2])
                        d['requester_num_pizza_related_comments_at_request'] = int(info_lines[2].split()[7])
                        
                        
#                 if "$fulfilled" in tl_comment.body:
#                     child_comments = []
#                     getSubComments(tl_comment, child_comments)
#                     for child_comment in child_comments:
#                         if "The transaction is now **CONFIRMED**" in child_comment.body:
#                             d['giver_username'] = tl_comment.author.name
#                             d['giver_user_flair'] = tl_comment.author_flair_text
#                             giver_redditor = tl_comment.author
                            
                            
                tl_text = tl_comment.body.lower()
                if "$fulfilled" in tl_text or "$confirm" in tl_text:
                    child_comments = []
                    getSubComments(tl_comment, child_comments)
                    for child_comment in child_comments:
                        if "I have changed the flair to ``In Progress``" in child_comment.body or "The transaction is now **CONFIRMED**" in child_comment.body:
                            info_lines = child_comment.body.splitlines() 
                            for info_line in info_lines:
                                if "**A**" in info_line:
                                    info_split = info_line.split("|")
                                    giver_redditor_name = info_split[2][3:]
                                    giver_redditor = reddit.redditor(giver_redditor_name)
                                    d['giver_username'] = giver_redditor_name
            except Exception as e:
                print("EXCPETION", e)
                continue
                    
            
        
        giver_subreddits_at_request = set()         
        if giver_redditor:
    
            for author_comment in giver_redditor.comments.top('all'):
                if author_comment.subreddit.created_utc < submission.created_utc:
                    giver_subreddits_at_request.add(author_comment.subreddit.display_name)
    
            for author_sub in giver_redditor.submissions.top('all'):
                if author_sub.subreddit.created_utc < submission.created_utc:
                    giver_subreddits_at_request.add(author_sub.subreddit.display_name)
                    
        
        d['giver_subreddits_at_request'] = list(giver_subreddits_at_request)
                
        d['requester_subreddits_at_request'] = list(requester_subreddits_at_request)
        
        d['requester_has_verified_email'] = submission.author.has_verified_email
    
        d['requester_received_pizza'] = 0
        if submission.link_flair_text == "Fulfilled":
            d['requester_received_pizza'] = 1
            
        d['requester_user_flair'] = submission.author_flair_text
        d['requester_username'] = submission.author.name
        d['unix_timestamp_of_request'] = submission.created_utc
        
        print(idx,submission.link_flair_text, d['giver_username'], d['request_url'])
        
        if d['requester_received_pizza']:
            if (d['giver_username'] == None):
                print ("WHYYY")
        
        
        return d
    

In [6]:
print (reddit.redditor("billyisnotthegoat").id)

4r32j7hi


In [85]:
def construct_features(idx): #index in submissions
        submission = submissions[idx]
        #print("STARTING...")
        d = {}

        d['request_id'] = submission.id
        d['request_number_of_comments_at_retrieval'] = submission.num_comments
        d['request_text'] = submission.selftext.lower()
        d['request_title'] = submission.title

        d['number_of_downvotes_of_request_at_retrieval'] = submission.downs
        d['number_of_upvotes_of_request_at_retrieval'] = submission.ups

        if submission.edited == False:
            d['post_was_edited'] = False
        else:
            d['post_was_edited'] = True
            
            
        

        d['requester_account_age_in_days_at_request'] = divmod(submission.created_utc - submission.author.created_utc, 86400)[0]

        d['request_url'] = submission.url


        try:
            requester_created_utc = submission.author.created_utc
        except: #if created_utc is not an attribute, the author has been suspended
            return


#         requester_subreddits_at_request = set()

#         for author_comment in submission.author.comments.top('all'):
#             if author_comment.subreddit.created_utc < submission.created_utc:
#                 requester_subreddits_at_request.add(author_comment.subreddit.display_name)

#         for author_sub in submission.author.submissions.top('all'):
#             if author_sub.subreddit.created_utc < submission.created_utc:
#                 requester_subreddits_at_request.add(author_sub.subreddit.display_name)

        top_level_comments = list(submission.comments)
    
        for tl_comment in top_level_comments:
            tl_text = tl_comment.body.lower()
            child_comments = []
            getSubComments(tl_comment, child_comments)
            for child_comment in child_comments:
                #print (child_comment.body) 
                if "**A**" in child_comment.body:
                    info_lines = child_comment.body.splitlines() 
                    for info_line in info_lines:
                        if "**A**" in info_line:
                            info_split = info_line.split("|")
                            giver_redditor_name = info_split[2][3:]
                            giver_redditor = reddit.redditor(giver_redditor_name)
                            #print (giver_redditor_name)



        d['requester_num_pizza_received_at_request'] = None
        d['requester_num_pizza_given_at_request'] = None
        d['requester_num_pizza_related_posts_at_request'] = None
        d['requester_num_pizza_related_comments_at_request'] = None

        d['giver_username'] = None
        d['giver_user_flair'] = None
        giver_redditor = None

        for tl_comment in top_level_comments:
            tl_text = tl_comment.body.lower()
            child_comments = []
            getSubComments(tl_comment, child_comments)
            for child_comment in child_comments:
                if "**A**" in child_comment.body:
                    info_lines = child_comment.body.splitlines()
                    for info_line in info_lines:
                        if "**A**" in info_line:
                            info_split = info_line.split("|")
                            giver_redditor_name = info_split[2][3:]
                            giver_redditor = reddit.redditor(giver_redditor_name)
                            d['giver_username'] = giver_redditor_name
            
            
            
            try:
                if tl_comment.author.id == GiversBotId:
                    if "* **Received" in tl_comment.body:
                        info_lines = tl_comment.body.splitlines()
                        d['requester_num_pizza_received_at_request'] = int(info_lines[0].split()[2] )
                        d['requester_num_pizza_given_at_request'] = int(info_lines[1].split()[2] )
                        d['requester_num_pizza_related_posts_at_request'] = int(info_lines[2].split()[2])
                        d['requester_num_pizza_related_comments_at_request'] = int(info_lines[2].split()[7])


                # tl_text = tl_comment.body.lower()
                # if "$fulfilled" in tl_text or "$confirm" in tl_text:
                #     child_comments = []
                #     getSubComments(tl_comment, child_comments)
                #     for child_comment in child_comments:
                #         if "I have changed the flair to ``In Progress``" in child_comment.body or "The transaction is now **CONFIRMED**" in child_comment.body:
                #             info_lines = child_comment.body.splitlines()
                #             for info_line in info_lines:
                #                 if "**A**" in info_line:
                #                     info_split = info_line.split("|")
                #                     giver_redditor_name = info_split[2][3:]
                #                     giver_redditor = reddit.redditor(giver_redditor_name)
                #                     d['giver_username'] = giver_redditor_name

                
            except Exception as e:
                print("EXCEPTION", e)
                continue



#         giver_subreddits_at_request = set()
#         if giver_redditor:

#             for author_comment in giver_redditor.comments.top('all'):
#                 if author_comment.subreddit.created_utc < submission.created_utc:
#                     giver_subreddits_at_request.add(author_comment.subreddit.display_name)

#             for author_sub in giver_redditor.submissions.top('all'):
#                 if author_sub.subreddit.created_utc < submission.created_utc:
#                     giver_subreddits_at_request.add(author_sub.subreddit.display_name)


#         d['giver_subreddits_at_request'] = list(giver_subreddits_at_request)

#         d['requester_subreddits_at_request'] = list(requester_subreddits_at_request)

#         d['requester_has_verified_email'] = submission.author.has_verified_email

#         d['requester_received_pizza'] = 0
#         if submission.link_flair_text == "Fulfilled":
#             d['requester_received_pizza'] = 1

#         d['requester_user_flair'] = submission.author_flair_text
#         d['requester_username'] = submission.author.name
#         d['unix_timestamp_of_request'] = submission.created_utc

        print(idx,submission.link_flair_text, d['giver_username'], d['request_url'])

        #if d['requester_received_pizza']:
        #    if (d['giver_username'] == None):
                #print ("WHYYY")


        return d

In [109]:
def test_feat(idx):
    print (idx)
    top_level_comments = list(submissions[idx].comments)
    
    print (submissions[idx].permalink)
    for tl_comment in top_level_comments:
        tl_text = tl_comment.body.lower()
        child_comments = []
        getSubComments(tl_comment, child_comments)
        for child_comment in child_comments:
            if "GIFT transaction #" in child_comment.body:
                info_lines = child_comment.body.splitlines() 
                #print (info_lines) 
                for info_line in info_lines:
                    if "**A**" in info_line:
                        info_split = info_line.split("|")
                        giver_redditor_name = info_split[2][3:]
                        giver_redditor = reddit.redditor(giver_redditor_name)
                        print (giver_redditor_name)

In [113]:
test_feat(745)

745
/r/Random_Acts_Of_Pizza/comments/bj80z5/request_wife_and_i_have_ate_about_10_pounds_of/
regula_et_vita
regula_et_vita


In [93]:
construct_features(1043)

IndexError: list index out of range

In [9]:
len(submissions)

4754

In [10]:
import multiprocessing
pool = multiprocessing.Pool(10)
dict_list = pool.map(construct_features, range(250))
pool.close()

STARTING... 7
STARTING... 21
STARTING... 14
STARTING... 28
STARTING... 35
STARTING... 42
STARTING... 49
STARTING... 0
STARTING... 56
STARTING... 63
EXCPETION 'NoneType' object has no attribute 'id'
28 None None https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dm33wy/request_hit_with_awful_charges_and_a_fine_broke/
STARTING... 29
14 None None https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dn76od/request_my_sibling_didnt_make_it_on_to_the_dance/
STARTING... 15
56 Fulfilled Evilevilcow https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dhh2ph/request_hungry_and_have_no_stove_oven_or_gas_to/
STARTING... 57
21 Fulfilled bobbyshark https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dmhdje/request_im_recovering_from_cervical_fusion/
STARTING... 22
42 Fulfilled bobbyshark https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/djdllj/request_burnt_dinner_out_of_cash/
STARTING... 43
49 Fulfilled Evilevilcow https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/di0j4g/reque

STARTING... 79
67 Fulfilled grapefruitdressgrape https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dg63w6/requestpoorer_than_a_hobo_in_a_barrel/
STARTING... 68
fetching (1 comments fetched total)
EXCPETION 'NoneType' object has no attribute 'id'
48 Fulfilled bobbyshark https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/di44hm/request_just_looking_for_a_little_extra_help_on/
STARTING... 84
26 Fulfilled Cmr017 https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dm71cz/request_no_paycheck_and_can_barely_walk_austin_tx/
STARTING... 27
EXCPETION 'NoneType' object has no attribute 'id'
79 None None https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dflhln/request_about_to_start_day_3_of_nothing_but_water/
STARTING... 80
EXCPETION 'NoneType' object has no attribute 'id'
68 None None https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dg54vy/request_my_sisterinlaws_husband_suddenly_passed/
STARTING... 69
72 Fulfilled Zeki_Boy https://www.reddit.com/r/Random_Acts_Of_Pizza/com

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 470, in _handle_results
    task = get()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
  File "/usr/local/lib/python3.7/site-packages/prawcore/exceptions.py", line 49, in __init__
    .format(response.status_code))
AttributeError: 'str' object has no attribute 'status_code'



EXCPETION 'NoneType' object has no attribute 'id'
119 Fulfilled unclesam4 https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/day9qb/request_pizza_for_me_please/
STARTING... 120
112 Fulfilled GSKermit https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dblcx4/request_college_student_broke_and_hungry_shocker/
STARTING... 113
92 Fulfilled Enacriel https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/demdyy/request_was_laid_off_two_weeks_ago_i_should_have/
STARTING... 93
105 Fulfilled procrastinator2112 https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dceuvp/request_would_love_a_pizza_in_these_trying_times/
STARTING... 106
75 Fulfilled procrastinator2112 https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dfpo1s/request_usa_studying_for_final_waiting_for/
STARTING... 76
140 None None https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/d8qcm1/requesttrying_to_cheer_up_my_girl_after_a_real/
STARTING... 141
120 None None https://www.reddit.com/r/Random_Acts_Of_Pizza/comme

STARTING... 162
150 Fulfilled MidniteProph https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/d7i8vs/request_feeling_a_bit_down_after_a_four_month/
STARTING... 151
124 Fulfilled toomanydeployments https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/damzuu/request_i_ruined_dinner_by_burning_it_beyond_all/
STARTING... 125
103 Fulfilled procrastinator2112 https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dcius8/request_disabled_wife_lost_their_job_today_pizza/
STARTING... 104
137 Fulfilled green_goblin_z https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/d9mgjs/request_unable_to_get_food_atm_need_help/
STARTING... 138
145 Fulfilled synthetic- https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/d7uoue/request_mugged_on_a_night_out_over_the_weekend/
STARTING... 146
EXCPETION 'NoneType' object has no attribute 'id'
EXCPETION 'NoneType' object has no attribute 'id'
97 Fulfilled Hiazi https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/ddv40b/request_unexpected_expense_le

...

KeyboardInterrupt: 

EXCPETION 'NoneType' object has no attribute 'id'
189 None None https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/d1yhpg/request_not_really_sure_how_this_works_but_pizza/
STARTING... 190
231 Fulfilled cbussthrowaway https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/cwmvat/request_29m_ohio_in_a_tough_spot/
STARTING... 232
224 None None https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/cxoyfo/request_exhausted_long_week_finally_got_a_job/
STARTING... 225
EXCPETION 'NoneType' object has no attribute 'id'
EXCPETION 'NoneType' object has no attribute 'id'
EXCPETION 'NoneType' object has no attribute 'id'
EXCPETION 'NoneType' object has no attribute 'id'
EXCPETION 'NoneType' object has no attribute 'id'
EXCPETION 'NoneType' object has no attribute 'id'
238 None None https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/cw5qvd/request_i_was_on_a_bus_that_crashed_last_night/
STARTING... 239
fetching (11 comments fetched total)
175 Fulfilled bobbyshark https://www.reddit.com/r/R

STARTING... 237
179 Fulfilled Sagerie https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/d3wfj4/request_awful_day_at_work_only_have_plain_rice_to/
STARTING... 180
EXCPETION 'NoneType' object has no attribute 'id'
199 Fulfilled dont_fuckup https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/d12eps/request_broke_couple_who_just_got_jobs_but_no_pay/
STARTING... 200
208 Fulfilled potatotron23 https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/d0fnqq/request_were_broke_and_almost_out_of_food_would/
STARTING... 209
EXCPETION 'NoneType' object has no attribute 'id'
221 Fulfilled bobbyshark https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/cy0mkg/request_kidney_stone_just_about_wiped_my_account/
STARTING... 222
EXCPETION 'NoneType' object has no attribute 'id'
187 Fulfilled AxlikeMike https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/d2gzmx/request_i_have_to_choose_between_keeping_the/
STARTING... 188
195 Fulfilled GizmosArrow https://www.reddit.com/r/Random_Acts_Of_Pizza/

Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Process ForkPoolWorker-19:
Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Process ForkPoolWorker-16:
Process ForkPoolWorker-20:
Process ForkPoolWorker-17:
Process ForkPoolWorker-14:
Process ForkPoolWorker-18:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 363, in put
    with self._wlock:
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 127, in worker
    put((job, i, result))
KeyboardInterrupt
  File "/u

In [11]:
df = pd.DataFrame(dict_list)


NameError: name 'dict_list' is not defined

In [20]:
len(dict_list)

20

In [64]:
df[["requester_received_pizza", "request_url"]]

,requester_received_pizza,request_url
0,0,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dpx5fe/request_i_have_4_to_get_me_through_the_next_day/
1,0,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dptmx0/request_hit_my_daily_limit_on_my_debit_card/
2,1,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dphb2r/request_really_stressful_week/
3,0,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dpe2dj/request_first_pay_day_of_new_job_is_friday_havent/
4,1,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dpccso/request_could_use_a_nice_pie_if_possible/
5,0,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dotuxd/request_cents_to_my_name_payday_is_tomorrow/
6,1,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dofjc3/request_vancouver_canada_lowincome_youth_looking/
7,1,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/dofige/request_37343_broke_no_car_and_no_food_in_my_house/
8,1,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/doeex3/request_im_struggling_right_now_even_pizza_i_cant/
9,1,https://www.reddit.com/r/Random_Acts_Of_Pizza/comments/doe5we/request_dad_of_3_would_really_like_the_boys_to/


In [ ]:
hm = pd.read_csv("raw_psaw.csv", sep = "\t")

In [ ]:
hm["selftext"][5]